# CS4320 Introduction to Machine Learning

**Please type your group name here:**


In [ ]:
GroupName = "NoProblem"
assert GroupName != "", 'Please enter your name in the above quotation marks, thanks!'

## Table of contents

1. [Understanding the problem](#1)
2. [Imports](#2)
   1. [Importing Packages](#2a)
   2. [Data splitting](#2b)
3. [EDA](#3)
4. [Feature engineering](#4)
5. [Preprocessing and transformations](#5)
6. [Baseline model](#6)
7. [Linear models](#7)
8. [Different models](#8)
9. [Feature selection](#9)
10. [Hyperparameter optimization](#10)
11. [Interpretation and feature importances](#11)
12. [Results on the test set](#12)
13. [Submit the predictions to Kaggle](#13)
14. [Your takeaway from the course](#14)


<!-- BEGIN QUESTION -->

## 1 Prediction Problem and Explaination <a name="1"></a>

<hr>

**Spend some time understanding the problem and what each feature means. Write a few sentences on your initial thoughts on the problem and the dataset.**


The problem is to forecast store sales for _Corporación Favorita_, a large Ecuadorian-based grocery retailer. The goal is to use previous sales data. The previously sales data is _family_ category which has a _store_nbr_, total number of items in that category that were _onpromotion_ that day, the specific date, and the total items sold that day for that family. The dates are able to be cross referenced with holidays in order to get a better time prediction. The family I am assuming will be very time dependent as needed change over the course of the year and many sales in my experience are holiday dependent.


<!-- END QUESTION -->

<br><br>


## 2 Imports <a name="2"></a>

<hr>


### 2a Importing Packages <a name="2a"></a>

<hr>

**Import all necessary packages**


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC

<!-- BEGIN QUESTION -->

### 2b Data splitting <a name="2b"></a>

<hr>

**Split the data into train and test portions. Note that the test.csv is the end file to test, not the data split for testing**


In [ ]:
dataset_df = pd.read_csv('data/train.csv')
final_test_df = pd.read_csv('data/test.csv')

In [ ]:
# split into train and test sets (in this case test_df is our validation set)
train_df, test_df = train_test_split(dataset_df, test_size=0.3, shuffle=False)

<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 3. EDA <a name="3"></a>

<hr>


### Perform exploratory data analysis on the train set, including summary statistics


In [ ]:
display(train_df.head())
display(train_df.info())

pd.set_option('display.float_format', '{:.2f}'.format)

display("Numerical Data Description")
display(train_df.describe(include=['int64', 'float64']))

display("Categorical Data Description")
display(train_df.describe(include=['object', 'datetime64'], datetime_is_numeric=True))

display("Family (Category) Counts")
display(train_df['family'].value_counts())

display("First and last date")
display(train_df['date'].min())
display(train_df['date'].max())

_Summary Statistics_

1. Here, we can see that there are 2 discrete varaibles: _date_ (1684 dates ranging from 2013 to 2016) and _family_ (33 unique family categories which have near equal statified counts).
2. The rest of the variables are numerical: id (unique identifier, discrete), store_nbr (1-54, discrete), sales (appears to be skewed, need to see visualization, continuous), and onpromotion (appears to be skewed, need to see visualization).


### Useful Visual Statistics


In [ ]:

eda_df = train_df.copy(deep=True)

# Create bar plot for 'date'
freq_df = eda_df['date'].value_counts().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='index', y='date', data=freq_df)
plt.title('Bar plot of date')
plt.xlabel('date')
plt.xticks(np.arange(0, len(freq_df['index']), step=200), rotation=90)
plt.text(0.5, -0.30, 'Figure 1', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.show()

# Create bar plot for 'store_nbr'
freq_df = eda_df['store_nbr'].value_counts().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='index', y='store_nbr', data=freq_df)
plt.title('Bar plot of store_nbr')
plt.xlabel('store_nbr')
plt.xticks(rotation=90)
plt.text(0.5, -0.15, 'Figure 2', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.show()

# Create bar plot for 'family'
freq_df = eda_df['family'].value_counts().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(x='index', y='family', data=freq_df)
plt.title('Bar plot of family')
plt.xlabel('family')
plt.xticks(rotation=90)
plt.text(0.5, -0.55, 'Figure 3', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.show()

In Figures 1,2, and 3, we are shown that _family_, _date_, and _store_nbr_ are all generally stratified. This is good as it means that we have a good distribution of data for each of these variables.


In [ ]:
# Show all sales 
plt.figure(figsize=(10, 6))
sns.histplot(train_df['sales'], bins=20)
plt.title('Histogram of Sales')
plt.xlabel('Sales', fontsize=12)
plt.text(0.5, -0.15, 'Figure 4', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.ylabel('Frequency')
plt.show()

# Show all onpromotion
plt.figure(figsize=(10, 6))
sns.histplot(train_df['onpromotion'], bins=20)
plt.title('Histogram of Onpromotion')
plt.xlabel('Onpromotion', fontsize=12)
plt.text(0.5, -0.15, 'Figure 5', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.ylabel('Frequency')
plt.show()

ONPROMOTION INVESTIGATION


In [ ]:
''' Test to see if Onpromotion should be kept'''
corr_matrix = train_df.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='Blues')
plt.title('Correlation Matrix')
plt.text(0.5, -0.15, 'Figure 6', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.show()

_onpromotion_ in Figure 6 has the highest correlation to sales, incidating it should be kept.


In [ ]:
''' We will test the Onpromotion variable, since it is an independent varaible, 
to see if Feature Cutting should be used. The feature cut would be the upper quantile'''

onpromo_zero_cnt = len(train_df[train_df['onpromotion'] == 0])
ratio = onpromo_zero_cnt / len(train_df)
print("Ratio of Zero in Onpromotion: {:.2f}".format(ratio))

In [ ]:
'''Since Ratio is too high, we will not use Feature Cutting and instead use ln(1+x) transformation to normalize the data, 
then use StandardScalar Feature Scaling in preprocessing. Until Feature Engineering, we will use the copy dateframe, eda_df''' 
eda_df['onpromotion'] = np.log1p(eda_df['onpromotion'])

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(eda_df['onpromotion'], bins=20)
plt.title('Histogram of Onpromotion')
plt.xlabel('Onpromotion', fontsize=12)
plt.text(0.5, -0.15, 'Figure 7', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.ylabel('Frequency')
plt.show()

SALES INVESTIGATION


In [ ]:
''' Test to see sales upper quantile significance '''

In [ ]:
''' For sales, since it is the dependent variable, Feature Cutting is not the first choice. Observing that the upper quantile of _sales_ is significant, we will apply a transformation to normalize the data, then use StandardScalar Feature Scaling in preprocessing'''
eda_df['sales'] = np.log1p(eda_df['sales'])

plt.figure(figsize=(10, 6))
sns.histplot(train_df['sales'], bins=20)
plt.title('Histogram of Sales')
plt.xlabel('Sales', fontsize=12)
plt.text(0.5, -0.15, 'Figure 8', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.ylabel('Frequency')
plt.show()

log(1+x) did not do much for _onpromotion_ and _sales_, showing that StandardScalar may not be the better scalar to use, and MinMax will need to be used.


In [ ]:
# Plot a histogram, seeing if some stores are more inclined to do promotions
promotion_store = train_df[train_df['onpromotion'] != 0].groupby('store_nbr')['onpromotion'].count().reset_index()
plt.figure(figsize=(10, 6))
ax = sns.barplot(x="store_nbr", y="onpromotion", data=promotion_store)
plt.title('Histogram of Onpromotion by Store Number')
plt.xlabel('Store Number')
plt.ylabel('Count of Promotions')
plt.xticks(rotation=90)
plt.text(0.5, -0.15, 'Figure 9', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.show()

The correlation matrix shows _store_nbr_ and _sales_ as not being correlated at all. It appears the _store_nbr_ is not a good predictor of _sales_, but _onpromotion_ is a good predictor of _sales_ and _onpromotion_ varies with _store_nbr_. So _store_nbr_ should be altered to a more relevent variable.


In [ ]:
# isolate year, month, and day from date
eda_df['date'] = pd.to_datetime(eda_df['date'])
eda_df['day'] = eda_df['date'].dt.strftime('%Y').astype(int)
eda_df['month'] = eda_df['date'].dt.strftime('%m').astype(int)
eda_df['year'] = eda_df['date'].dt.strftime('%d').astype(int)
eda_df = eda_df.drop(columns=['date'])
eda_df

In [ ]:
eda_df

In [ ]:
corr_matrix = eda_df.drop(columns=['family', 'id']).corr()
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='Blues')
plt.title('Correlation Matrix')
plt.text(0.5, -0.15, 'Figure 10', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')
plt.show()

- Since this is a time-series problem, we will be putting a heavier emphasis on the _date_ variable by separating it into day, month, and year. Above, you can see it performed on the _eda_df_ which has been used for testing data augmentation without impacting the original dataset. Now, these changes can be applied in the feature engineering step., so that _date_ can be made more useful. Observe the .15 correlation between _day_ and _sales_. _store_nbr_ is still low and will be changed. _month_ and _year_ are also low, but will be kept for now.


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 4. Feature engineering <a name="4"></a>

<hr>

**Your tasks:**

1. Carry out feature engineering. In other words, extract new features relevant for the problem and work with your new feature set. You may have to go back and forth between feature engineering and preprocessing. Briefly explain why you come up with these new features.


In [ ]:
# Apply sales and onpromotion transformation
train_df['onpromotion'] = np.log1p(train_df['onpromotion'])
train_df['sales'] = np.log1p(train_df['sales'])

In [ ]:
# Import useful CSV'set
holiday_events_df = pd.read_csv('data/holidays_events.csv')
stores_df = pd.read_csv('data/stores.csv')

In [ ]:
# Show the imported CVS's head
display(holiday_events_df.head())
display(stores_df.head())

In [ ]:
# Separate useful df's into smaller df's
national_holiday_df = holiday_events_df[(holiday_events_df['locale'] == 'National') & (holiday_events_df['type'] == 'Holiday')]['date']
national_not_holiday_df = holiday_events_df[(holiday_events_df['locale'] == 'National') & (holiday_events_df['type'] != 'Holiday')]['date']

local_holiday_df = holiday_events_df[(holiday_events_df['locale'] == 'Local') & (holiday_events_df['type'] == 'Holiday')]['date']
local_not_holiday_df = holiday_events_df[(holiday_events_df['locale'] == 'Local') & (holiday_events_df['type'] != 'Holiday')]['date']

In [ ]:
def holiday_transform(dataframe):
    dataframe['national_holiday'] = dataframe['date'].isin(national_holiday_df).astype(int)
    dataframe['national_not_holiday'] = dataframe['date'].isin(national_not_holiday_df).astype(int)
    dataframe['local_holiday'] = dataframe['date'].isin(local_holiday_df).astype(int)
    dataframe['local_not_holiday'] = dataframe['date'].isin(local_not_holiday_df).astype(int)
    return dataframe

def date_transform(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['day'] = dataframe['date'].dt.strftime('%Y')
    dataframe['month'] = dataframe['date'].dt.strftime('%m')
    dataframe['year'] = dataframe['date'].dt.strftime('%d')
    dataframe = dataframe.drop(columns=['date'])
    return dataframe

In [ ]:
train_df = date_transform(holiday_transform((train_df)))
train_df.head()

In [ ]:
# transform 'store_nbr' based on what cluster, city, and state it is in
def store_transform(dataframe):
    dataframe['store_cluster'] = stores_df['cluster'].values[dataframe['store_nbr'].values - 1]
    dataframe['city'] = stores_df['city'].values[dataframe['store_nbr'].values - 1]
    dataframe['state'] = stores_df['state'].values[dataframe['store_nbr'].values - 1]
    dataframe = dataframe.drop(columns=['store_nbr'])
    return dataframe

train_df = store_transform(train_df)

In [ ]:
# Display manual transformations so far
train_df.head()

In [ ]:
''' Quick Feature Test'''
corr_matrix = train_df.corr()
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='Blues')
plt.title('Correlation Matrix')
plt.text(0.5, -0.5, 'Figure 11', transform=plt.gca().transAxes, ha='center', va='center', fontsize=12, style='italic')

plt.show()

### Separating Feature Vectors and Targets


In [ ]:
X_train = train_df.drop(columns=['sales'])
y_train = train_df['sales']

<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 5. Preprocessing and transformations <a name="5"></a>


### Identifying different feature types and the transformations to apply on each feature type.


In [ ]:
# Feature List
categorical_features = ['family', 'store_cluster', 'city', 'state'] # One Hot Encoded
binary_features = ['national_holiday', 'national_not_holiday', 'local_holiday', 'local_not_holiday'] # One Hot Encoded (Binary)
scaling_features = ['onpromotion', 'day', 'month', 'year'] # MinMax Scaled
drop_features = ['id'] # Drop
target = 'sales'

### Defining a column transformer and perform preprocessing on the train set


In [ ]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
binary_transformer = OneHotEncoder(drop="if_binary", dtype=int)
scaling_transformer = StandardScaler()

preprocessor = make_column_transformer(
    (categorical_transformer, categorical_features),
    (binary_transformer, binary_features),
    (scaling_transformer, scaling_features),
    ('drop', drop_features),
    remainder='passthrough'
)

preprocessor

In [ ]:
X_train_prepared = preprocessor.fit_transform(X_train)
X_train_prepared

In [ ]:
column_names = list(
    preprocessor.named_transformers_["onehotencoder-1"].get_feature_names_out(categorical_features)) + list(
    preprocessor.named_transformers_["onehotencoder-2"].get_feature_names_out(binary_features)) + scaling_features

column_names

In [ ]:
X_trained_transformed = pd.DataFrame(X_train_prepared, columns=column_names)
X_trained_transformed

<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 6. Baseline model <a name="6"></a>

<hr>

**Your tasks:**

1. Try `scikit-learn`'s baseline model and report results.


In [ ]:
X_test = test_df.drop(columns=['sales'])
y_test = test_df['sales']

y_test = np.log1p(y_test)
X_test['onpromotion'] = np.log1p(X_test['onpromotion'])

X_test_prep = store_transform(date_transform(holiday_transform((X_test))))
X_test_prepared = preprocessor.transform(X_test_prep)
X_test_transformed = pd.DataFrame(X_test_prepared, columns=column_names)

In [ ]:
# Use skikit learn baseline model for numerical variables
dummy_reg = DummyRegressor(strategy="mean")
dummy_reg.fit(X_trained_transformed, y_train)
dummy_reg.score(X_test_transformed, y_test)

<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 7. Linear models <a name="7"></a>

<hr>

**Your tasks:**

1. Try a linear model as a first real attempt.
2. Carry out hyperparameter tuning to explore different values for the complexity hyperparameter.
3. Report cross-validation scores along with standard deviation.
4. Summarize your results.


In [ ]:
# Step 1: Create a model
model = Ridge()
model.fit(X_trained_transformed, y_train)
model.score(X_test_transformed, y_test)

In [ ]:
# Step 2: Hyperparameter tuning
# Define the parameter grid
param_grid = {'alpha': np.arange(0, 5, 0.1)}

# Initialize the Randsearch object
rand_search = RandomizedSearchCV(model, param_grid, cv=5, n_iter=20)

# Fit the RandomizedSearchCV object to the data
rand_search.fit(X_trained_transformed, y_train)

In [ ]:
print("Best params: ", rand_search.best_params_)

print("Best score: ", rand_search.best_score_)

pd.DataFrame(rand_search.cv_results_)[
    [
        "mean_test_score",
        "param_alpha",
        "mean_fit_time",
        "rank_test_score",
    ]
].set_index("rank_test_score").sort_index().T

In [ ]:
# cross validate with best params
best_alpha = rand_search.best_params_.get('alpha')
model = Ridge(alpha=best_alpha)
cv_results = cross_validate(model, X_trained_transformed, y_train, cv=10, scoring='r2')
print("CV Mean: ", cv_results['test_score'].mean())
print("CV Std Dev: ", cv_results['test_score'].std())

In [ ]:
pd.DataFrame(cv_results)

The linear model is a good first attempt as it is simple and easy to understand and the standard deviation is very low. The hyperparameter tuning has cross validations scores that are lower than the inital ridge regression default. Overall this insinuates that the linear model might be a good fit for the data.


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 8. Different models <a name="8"></a>

<hr>

**Your tasks:**

1. Try other models aside from a linear model. One of these models should be a tree-based ensemble model.
2. Summarize your results in terms of overfitting/underfitting and fit and score times. Can you beat a linear model?


In [ ]:
model_rfr = RandomForestRegressor(max_depth=10, n_estimators=15)
#model_rfr.fit(X_trained_transformed, y_train)
#model_rfr.score(X_test_transformed, y_test)
rfr_cv = cross_validate(model_rfr, X_trained_transformed, y_train, scoring='r2', cv=3)
print("CV Mean: ", rfr_cv['test_score'].mean())
print("CV Std Dev: ", rfr_cv['test_score'].std())

In [ ]:
model_dtr = DecisionTreeRegressor(max_depth=10)
#model_dtr.fit(X_trained_transformed, y_train)
#model_dtr.score(X_test_transformed, y_test)
dtr_cv = cross_validate(model_dtr, X_trained_transformed, y_train, scoring='r2', cv=3)
print("CV Mean: ", dtr_cv['test_score'].mean())
print("CV Std Dev: ", dtr_cv['test_score'].std())

In [ ]:
import xgboost as x
model_xg = x.XGBRegressor()
#model_xg.fit(X_trained_transformed, y_train)
#model_xg.score(X_test_transformed, y_test)
xg_cv = cross_validate(model_xg, X_trained_transformed, y_train, scoring='r2', cv=3)
print("CV Mean: ", xg_cv['test_score'].mean())
print("CV Std Dev: ", xg_cv['test_score'].std())

_Type your answer here, replacing this text._


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 9. Feature selection <a name="9"></a>

<hr>

**Your tasks:**

Make some attempts to select relevant features. Do the results improve with feature selection? Summarize your results. If you see improvements in the results, keep feature selection in your pipeline. If not, you may abandon it.


In [ ]:
lr_1 = make_pipeline(preprocessor, Ridge(alpha=1))
lr_fitted_1 = lr_1.fit(X_train, y_train)
x_train_enc_1 = lr_fitted_1.named_steps['columntransformer'].transform(X_train)


In [ ]:
# Show multiple correlation matrices graphs, 4x2 grid
fig, axs = plt.subplots(4, 2, figsize=(20, 20))
cor_1 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2)), axis=1).iloc[:, :10].corr()
cor_2 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 11:20]), axis=1).iloc[:, :10].corr()
cor_3 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 21:30]), axis=1).iloc[:, :10].corr()
cor_4 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 31:40]), axis=1).iloc[:, :10].corr()
cor_5 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 41:50]), axis=1).iloc[:, :10].corr()
cor_6 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 51:60]), axis=1).iloc[:, :10].corr()
cor_7 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 61:70]), axis=1).iloc[:, :10].corr()
cor_8 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 71:80]), axis=1).iloc[:, :10].corr()

sns.heatmap(cor_1, ax=axs[0, 0], cmap='Blues', annot=True)
sns.heatmap(cor_2, ax=axs[0, 1], cmap='Blues', annot=True)
sns.heatmap(cor_3, ax=axs[1, 0], cmap='Blues', annot=True)
sns.heatmap(cor_4, ax=axs[1, 1], cmap='Blues', annot=True)
sns.heatmap(cor_5, ax=axs[2, 0], cmap='Blues', annot=True)
sns.heatmap(cor_6, ax=axs[2, 1], cmap='Blues', annot=True)
sns.heatmap(cor_7, ax=axs[3, 0], cmap='Blues', annot=True)
sns.heatmap(cor_8, ax=axs[3, 1], cmap='Blues', annot=True)

plt.tight_layout()
plt.show()



In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))
cor_9 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 81:90]), axis=1).iloc[:, :10].corr()
cor_10 = pd.concat((pd.Series(y_train), pd.DataFrame(x_train_enc_2).iloc[:, 91:100]), axis=1).iloc[:, :10].corr()

sns.heatmap(cor_9, ax=axs[0], cmap='Blues', annot=True)
sns.heatmap(cor_10, ax=axs[1], cmap='Blues', annot=True)

# plt.tight_layout()
plt.show()


In [ ]:
# Rest removing 'year'
preprocessor_2 = make_column_transformer(
    (categorical_transformer, categorical_features),
    (binary_transformer, binary_features),
    (scaling_transformer, scaling_features[:-1]),
    ('drop', drop_features),
    remainder='passthrough'
)

lr_2 = make_pipeline(preprocessor_2, KNeighborsRegressor(n_neighbors=5))
lr_fitted_2 = lr_2.fit(X_train_dropped, y_train)
x_train_enc_2 = lr_fitted_2.named_steps['columntransformer'].transform(X_train_dropped)
pred_2 = lr_fitted_2.predict(x_test_without_year)

In [ ]:
best_feats = ['family_GROCERY I',
'family_BEVERAGES',
'family_CLEANING',
'family_DAIRY',
'family_BREAD/BAKERY',
'family_MEATS',
'family_POULTRY',
'family_PERSONAL CARE',
'family_DELI',
'family_PET SUPPLIES',
'family_HARDWARE',
'family_MAGAZINES',
'family_HOME APPLIANCES',
'family_SCHOOL AND OFFICE SUPPLIES',
'family_BABY CARE',
'family_BOOKS']

X_feats = X_trained_transformed[best_feats]
X_feats_test = X_test_transformed[best_feats]
feats_boost = x.XGBRegressor()

feats_boost.fit(X_feats, y_train)
feats_scores = cross_validate(feats_boost, X_feats_test, y_test, cv=3, scoring='r2')
print("CV Mean: ", feats_scores['test_score'].mean())
print("CV Std Dev: ", feats_scores['test_score'].std())

Clearly, eliminating features with lower coefficients isn't ideal here. This could be because of the sheer number of features created, since the sum total of many small coefficients can make a big difference. At least for a reasonably fast model like XGB or random forest, the trade off for eliminating features with coefficients below 2.0 is probably not worth it, despite the runtime improvement.


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 10. Hyperparameter optimization <a name="10"></a>

<hr>

**Your tasks:**

Make some attempts to optimize hyperparameters for the models you've tried and summarize your results. In at least one case you should be optimizing multiple hyperparameters for a single model. You may use `sklearn`'s methods for hyperparameter optimization or fancier Bayesian optimization methods.

- [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
- [RandomizedSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
- [scikit-optimize](https://github.com/scikit-optimize/scikit-optimize)


In [ ]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform, randint
import xgboost as x

def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


params = { # For XGBoost
    "learning_rate": uniform(0.08, 0.13),
    "max_depth": randint(2, 6),
    "n_estimators": randint(80, 120),
    "subsample": uniform(0.6, 0.4)
}

model = x.XGBRegressor()

# params = {
#     "max_depth": randint(2, 6),
#     "n_estimators": randint(20, 30)
# }

# model = RandomForestRegressor(random_state=42, )


search = RandomizedSearchCV(model, param_distributions=params, random_state=42, n_iter=10, cv=3, verbose=1, n_jobs=1, return_train_score=True, scoring='r2')

In [ ]:
search.fit(X_trained_transformed, y_train)

report_best_scores(search.cv_results_, 3)

# XGBOOST HYPERPARAMETER TUNING RESULTS:
# Model with rank: 1
# Mean validation score: 0.709 (std: 0.035)
# Parameters: {'learning_rate': 0.15782560294561476, 'max_depth': 4, 'n_estimators': 98, 'subsample': 0.6399899663272012}

# Model with rank: 2
# Mean validation score: 0.698 (std: 0.038)
# Parameters: {'learning_rate': 0.20201185217204753, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8469926038510867}

# Model with rank: 3
# Mean validation score: 0.689 (std: 0.037)
# Parameters: {'learning_rate': 0.20248622823903087, 'max_depth': 3, 'n_estimators': 88, 'subsample': 0.6063865008880857}

# RANDOM FOREST HYPERPARAMETER TUNING RESULTS: 
# Fitting 2 folds for each of 5 candidates, totalling 10 fits
# Model with rank: 1
# Mean validation score: 0.155 (std: 0.045)
# Parameters: {'max_depth': 4, 'n_estimators': 23}

# Model with rank: 2
# Mean validation score: 0.155 (std: 0.046)
# Parameters: {'max_depth': 4, 'n_estimators': 26}

# Model with rank: 3
# Mean validation score: 0.154 (std: 0.046)
# Parameters: {'max_depth': 4, 'n_estimators': 29}

_Type your answer here, replacing this text._


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 11. Interpretation and feature importances <a name="11"></a>

<hr>

**Your tasks:**

1. Use the methods we saw in class (e.g., `eli5`, `shap`) (or any other methods of your choice) to examine the most important features of one of the non-linear models.
2. Summarize your observations.


In [ ]:
# Your code here
X_train_enc = pd.DataFrame(
    data=preprocessor.fit_transform(X_train),
    columns=column_names,
    index=X_train.index,
)
X_train_enc.head()

In [ ]:
import shap
shap.initjs()

y_test_reset = y_test.reset_index(drop=True)
y_test_reset

In [ ]:
# Update here, file 15 

_Type your answer here, replacing this text._


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 12. Results on the test set <a name="12"></a>

<hr>

**Your tasks:**

1. Try your best performing model on the test data (from train test split) and report test scores.
2. Do the test scores agree with the validation scores from before? To what extent do you trust your results? Do you think you've had issues with optimization bias?
3. Take one or two test predictions and explain these individual predictions (e.g., with SHAP force plots).


In [ ]:
# Your code here
model_xg.fit(x_trained_transformed, y_train)
best_scores = model_xg.score(x_test_transformed, y_test)
print(best_scores)

The R2 score of the xgboost model seems to perform even better on the test set than the validation partitions. I suspect this is due to the validation sets being highly skewed, since even powerful ensemble models like random forest seemed to struggle with CV scores. Also, our final dataframe has a somewhat large amount of fields, so it is possible that there is very high variance within the validation sets, and that is also seen in the somewhat high standard deviations of CV scores.


_Type your answer here, replacing this text._


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 13. Submit the predictions to Kaggle <a name="13"></a>

<hr>

**Your tasks:**

Retrain the best model on the whole training dataset and upload the predicted output on the test set to Kaggle. Report your final test score.


In [ ]:
# Your code here
best_model = [-----]
best_model.fit(X_train_prepared, y_train)

y_pred = model.predict(final_test_df)

results = pd.concat([test_df['id'], pd.Series(y_pred)], axis=1)
results.columns = ['id', 'sales']
results.to_csv('data/submission2_2.csv', index = False, header=True)

results

_Type your answer here, replacing this text._


<!-- END QUESTION -->

<br><br>


<!-- BEGIN QUESTION -->

## 14. Your takeaway <a name="14"></a>

<hr>

**Your tasks:**

What is your biggest takeaway from the supervised machine learning material we have learned so far? Please write thoughtful answers. Discuss other ideas that you did not try but could potentially improve the performance/interpretability .


<!-- END QUESTION -->

<br><br>
